In [36]:
function inexact_method(f, g, p0, pd0, xk, dk; t=0.5, e=0.5, c=2)
    a = 1
    da=dk'*g((xk .+ a.*dk)...)
    while da <0
        a = c*a
        da=dk'*g((xk .+ a .*dk)...)
    end
    pa=f((xk .+ a .* dk)...)
    while(pa>p0+e*a*pd0)
        a = t*a
        pa=f((xk .+ a .* dk)...)
    end
    return a
end

inexact_method (generic function with 1 method)

In [75]:
#####################梯度下降法#######################
function gradient_descent(f, g, x0; a0=0.1,
        eg=0.0001, ex=0.0001, ef=0.0001, maxIterations=128, verbose=true)
    x1 = x0
    d  = -g(x1...)
    f1 = f(x1...)
    ng = norm(d)
    # inexact method for a
    a  = inexact_method(f, g, f1, -ng'*ng, x1, d)
    x2 = x1 + a .* d
    f2 = f(x2...)
    nx = a*ng
    ni = 1
    while( ng>eg || nx>ex || abs(f2-f1)>ef && ni<maxIterations)
        x1 = x2
        d  =-g(x1...)
        ng = norm(d)
        f1 = f2
        a  = inexact_method(f, g, f1, -ng'*ng, x1, d)
        x2 = x1 + a .* d
        f2 = f(x2...)
        nx = a*ng
        ni += 1
        if verbose
            println("Step ",ni, " x ", x2 ," f ",f2)
            println("      d",d ," a ",a)
        end
    end
    if ni >= maxIterations
        println("WARNING: Iterations exceeds ",maxIterations)
    else
        println("LOG:iteration converges after", ni, "step")
    end
    return x2, f2
end

gradient_descent (generic function with 1 method)

In [76]:
gradient_descent(
    (x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2 ; 200y-200x^2],
    [-2 2]', eg=0.0001, ex=0.0001, ef=0.0001, maxIterations=128, verbose=false)

([0.999898; 0.999795], 1.046059921951776e-8)

In [1]:
###################牛顿法####################
function Newton( f, g, h, x0, e=0.0001, verbose=true)
    x1=x0
    g1=g(x1...)
    ni=0
    f1=f(x1...)
    while(norm(g1)>=e)
        h1=h(x1...)
        if norm(h1)>0
            x2=x1-inv(h1)*g1
            x1=x2
            f1=f(x1...)
            g1=g(x1...)
            ni +=1
            if verbose
                println("Step ",ni, " x= ", x1 )
                println(" f= ",f1," g= ",g1)
            end 
            else println("H is not positive definite matrix.")
        end
    end
    return x1,f1,g1
end

Newton (generic function with 3 methods)

In [2]:
Newton(
    (x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2 ; 200y-200x^2],
    (x,y)->[2+800x^2-400(y-x^2) -400x; -400x 200]',
    [-2 2]'
)

Step 1 x= [-1.99252; 3.97007]
 f= 8.955168502514422 g= [-6.02965, -0.011194]
Step 2 x= [0.966873; -7.82315]
 f= 7670.252979776174 g= [3387.08, -1751.6]
Step 3 x= [0.966892; 0.934879]
 f= 0.001096166665157438 g= [-0.0662167, -7.14555e-8]
Step 4 x= [1.0; 0.998904]
 f= 0.0001201581014363651 g= [0.438467, -0.219233]
Step 5 x= [1.0; 1.0]
 f= 1.8097993700064961e-19 g= [-8.50746e-10, -5.68434e-14]


([1.0; 1.0], 1.8097993700064961e-19, [-8.50746e-10, -5.68434e-14])

In [ ]:
x=1
y=2
[2(x-1)-400*x*(y-x^2) 200(y-x^2)]'

In [3]:
############Levenberg_Marquardt改进算法################
function Levenberg_Marquardt( f, g, h, x0, r,e=0.0001, verbose=true)
    x1=x0
    g1=g(x1...)
    ni=0
    f1=f(x1...)
    while(norm(g1)>=e)
        h1=h(x1...)
        if norm(h1)>0
            x2=x1-inv(h1)*g1
            x1=x2
            f1=f(x1...)
            g1=g(x1...)
            ni +=1
            if verbose
                println("Step ",ni, " x= ", x1 )
                println(" f= ",f1," g= ",g1)
            end 
        else 
            x2=x1-inv(h1+r*eye(h1))*g1
            x1=x2
            f1=f(x1...)
            g1=g(x1...)
            ni+=1
            if verbose
                println("Step ",ni, " x= ", x1 )
                println(" f= ",f1," g= ",g1)
            end
        end
    end
    return x1,f1,g1
end

Levenberg_Marquardt (generic function with 3 methods)

In [4]:
Levenberg_Marquardt(
    (x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2 ; 200y-200x^2],
    (x,y)->[2+800x^2-400(y-x^2) -400x; -400x 200]',
    [-2 2]',
    100000
)

Step 1 x= [-1.99252; 3.97007]
 f= 8.955168502514422 g= [-6.02965, -0.011194]
Step 2 x= [0.966873; -7.82315]
 f= 7670.252979776174 g= [3387.08, -1751.6]
Step 3 x= [0.966892; 0.934879]
 f= 0.001096166665157438 g= [-0.0662167, -7.14555e-8]
Step 4 x= [1.0; 0.998904]
 f= 0.0001201581014363651 g= [0.438467, -0.219233]
Step 5 x= [1.0; 1.0]
 f= 1.8097993700064961e-19 g= [-8.50746e-10, -5.68434e-14]


([1.0; 1.0], 1.8097993700064961e-19, [-8.50746e-10, -5.68434e-14])